Section 1: Loading in the data

1. Import the required packages needed for execution of the code.

In [ ]:
#conda packages
from matplotlib import pyplot as plt
from matplotlib import colormaps as cm
from matplotlib import patches
import numpy as np
import datetime
import pickle
from obspy import Trace
from obspy.signal.trigger import classic_sta_lta
from sklearn.neighbors import NearestNeighbors
from tqdm.notebook import tqdm_notebook as tqdm
from sklearn.cluster import HDBSCAN

#project imports
from TDMS_Read import TdmsReader #code provided by Silixa
from TDMS_Utilities import scale, get_data
from filters import filter_waterfall
from SSTA import ssta
from clusterAndClassify import plot_clusters

2. Load in the data for one of the three provided windows (Uncomment as necessary):
   - November - 1000 Hz, 10 seconds 
   - January  - 1000 Hz, 10 seconds
   - February - 1000 Hz, 30 seconds, paper uses 5 secs - 15 secs

In [ ]:
directory = "Example Windows" 

#November 2023

file_path = f"{directory}/November_Window_UTC_20231109_134947.573.tdms" #rockfall + nano
first_time_sample = 0
last_time_sample = 10000 - 1

#January 2024

#file_path = f"{directory}/January_Window_UTC_20240117_130137.656.tdms" #forest noise and potential nano?
#first_time_sample = 0
#last_time_sample = 10000 - 1

#February 2024
# file_path = f"{directory}/February_Window_UTC_20240227_103823.084.tdms" # walker, potenital nano
# first_time_sample = 5000
# last_time_sample = 15000 - 1

In [ ]:
#loads up file and extracts metadata

print('File: {0}'.format(file_path))

tdms = TdmsReader(file_path)

props = tdms.get_properties()

#where does data recording start
zero_offset = props.get('Zero Offset (m)')
#where does each channel sit along the cable
channel_spacing = props.get('SpatialResolution[m]') * props.get('Fibre Length Multiplier')
#how many channels are there
n_channels = tdms.fileinfo['n_channels']
#distance along the cable called depth here but hey
depth = zero_offset + np.arange(n_channels) * channel_spacing
#sampling frequency
fs = props.get('SamplingFrequency[Hz]')
time = props.get('GPSTimeStamp')

print('Number of channels in file: {0}'.format(n_channels))
print('Time samples in file: {0}'.format(tdms.channel_length))
print('Sampling frequency (Hz): {0}'.format(fs))
print(f'Time of Recording: {time}')
first_channel = 0
last_channel = n_channels

#load in data
some_data = tdms.get_data(first_channel, last_channel, first_time_sample, last_time_sample)
print('Size of data loaded: {0}'.format(some_data.shape))

In [ ]:
# 𝑠𝑡𝑟𝑎𝑖𝑛𝑟𝑎𝑡𝑒 𝑛𝑚 𝑚𝑠 = 116 𝑥 𝑖𝐷𝐴𝑆 𝑣𝑎𝑙𝑢𝑒𝑠 𝑥 𝑠𝑎𝑚𝑝𝑙𝑖𝑛𝑔 𝑓𝑟𝑒𝑞 (𝐻𝑧) 𝑔𝑎𝑢𝑔𝑒 𝑙𝑒𝑛𝑔𝑡ℎ (𝑚)
some_data = scale(some_data, props)
first_time_sample = 0
last_time_sample = 10000 - 1

3. Plot Loaded File

In [ ]:
bounds = 1000

plt.set_cmap(plt.cm.get_cmap('bwr'))
fig, ax = plt.subplots()
img1 = ax.imshow(some_data, aspect='auto', interpolation='none', vmin=-bounds, vmax=bounds, extent=(first_channel,last_channel-1, ((last_time_sample - 1)/fs), (first_time_sample/fs)))

#location of different events of interest
#Jan
#plt.xlim(1500, 1700)

#plt.xlim(2500, 2800)
#plt.ylim(3.4, 3.8)

#Feb

#plt.xlim(2840, 2920)
#plt.ylim(10, 9)

#Nov
#plt.ylim(10, 0)

#plt.xlim(1750, 1820)
#plt.ylim(9, 8)

#plt.xlim(5000, 7000)
#plt.ylim(10, 6)

plt.ylabel('Time (seconds)')
plt.xlabel('Distance (Channels)')
plt.title(f"November Day - Footsteps - {props.get('GPSTimeStamp')}\nRaw Waterfall")

fig.colorbar(img1, label= "Nano Strain per Second [nm/m/s]")

plt.show(block=False)

Section 2: Filter the Data

1. High-pass filter the data at 100Hz

In [ ]:
highcut = -1
lowcut = 100
filtered_data = filter_waterfall(some_data, fs, lowcut=lowcut, highcut=highcut)

2. Plot the data

In [ ]:
bounds = 1000

fig, ax = plt.subplots()
img1 = ax.imshow(filtered_data, aspect='auto', interpolation='none', vmin=-bounds, vmax=bounds, extent=(first_channel,last_channel-1, ((last_time_sample - 1)/fs), (first_time_sample/fs)))

#location of different events of interest
#Jan
#plt.xlim(1500, 1700)

#plt.xlim(2500, 2800)
#plt.ylim(3.4, 3.8)

#Feb

#plt.xlim(2840, 2920)
#plt.ylim(10, 9)

#Nov
#plt.ylim(10, 0)

#plt.xlim(1750, 1820)
#plt.ylim(9, 8)

#plt.xlim(5000, 7000)
#plt.ylim(10, 6)

plt.ylabel('Time (seconds)')
plt.xlabel('Distance (Channels)')
plt.title(f"January Day - Footsteps - {props.get('GPSTimeStamp')}\n100Hz High-pass Filtered Waterfall")

plt.set_cmap(plt.cm.get_cmap('bwr'))
fig.colorbar(img1, label= "Nano Strain per Second [nm/m/s]")


plt.show(block=False)

Section 3: Method Comparison

1. Select data to compare (filtered or unfiltered) and region of interest

In [ ]:
data = filtered_data
filtered = True

#data = some_data
#filtered = False

#region of interest min and max channel
start_channel = 1650
end_channel = 8650

2. Process data using event detection method

- SSTA

In [ ]:
#number of time samples in window
num = 2

#threshold for anomalous amplitudes
threshold = 8

#process data and create array of locations of anomalous points
mask = ssta(data, start_channel=start_channel, end_channel=end_channel, thresh=threshold, num=num)
SSTA_Points = np.where(mask == 1)
print(SSTA_Points)

3. Plot extracted anomalous points

In [ ]:
bounds = 1000
fig, ax = plt.subplots()
img1 = ax.imshow(data, aspect='auto', interpolation='none', vmin=-bounds, vmax=bounds, extent=(first_channel,last_channel-1, ((last_time_sample - 1)/fs), (first_time_sample/fs)))
plt.set_cmap(cm.get_cmap('bwr'))

ax.plot(SSTA_Points[1], SSTA_Points[0]/fs, color='black', linewidth = 3, marker='o', markerfacecolor='red', markersize=1, linestyle='None', label="Anomaly")

plt.ylabel('Time (seconds)')
plt.xlabel('Distance (Channels)')
fig.colorbar(img1, label= "Nano Strain per Second [nm/m/s]")

#location of different events of interest
#Jan
#plt.xlim(1500, 1700)

#plt.xlim(2500, 2800)
#plt.ylim(3.4, 3.8)

#Feb

#plt.xlim(2840, 2920)
#plt.ylim(10, 9)

#Nov
#plt.ylim(10, 0)

#plt.xlim(1750, 1820)
#plt.ylim(9, 8)

#plt.xlim(5000, 7000)
#plt.ylim(10, 6)

plt.show(block=False)
print(len(SSTA_Points[0]))

4. Repeat process for other methods

- STA/LTA

In [ ]:
lta = 0.2
sta = 0.002
threshold = 16

processed_tdms = np.empty((last_time_sample+1, last_channel), type(data[0, 0]))

with tqdm(total=len(data[0])) as pbar:
    for samp_num in range(0, len(data[0])):
        #print(samp_num)
        if start_channel <= samp_num <= end_channel:
            time_sample = data.transpose()[samp_num,:]
            trace = Trace(np.array(time_sample))

            filtered_signal = classic_sta_lta(time_sample, int(sta * fs), int(lta * fs))

            for i, point in enumerate(filtered_signal):
                processed_tdms[i, samp_num] = point

        pbar.update()

STALTA_Points = np.where(processed_tdms > threshold)
print(STALTA_Points)

In [ ]:
bounds = 1000
plt.set_cmap(cm.get_cmap('bwr'))
fig, ax = plt.subplots()
img1 = ax.imshow(data, aspect='auto', interpolation='none', vmin=-bounds, vmax=bounds, extent=(first_channel,last_channel-1, ((last_time_sample - 1)/fs), (first_time_sample/fs)))
#ax.set_xlim(1200, 1500)
plt.ylabel('Time (seconds)')

ax.plot(STALTA_Points[1], STALTA_Points[0]/fs, color='black', linewidth = 3, marker='o', markerfacecolor='black', markersize=1, linestyle='None', label="Anomaly")

plt.ylabel('Time (seconds)')
plt.xlabel('Distance (Channels)')
fig.colorbar(img1, label= "Nano Strain per Second [nm/m/s]")

#location of different events of interest
#Jan
#plt.xlim(1500, 1700)

#plt.xlim(2500, 2800)
#plt.ylim(3.4, 3.8)

#Feb

#plt.xlim(2840, 2920)
#plt.ylim(10, 9)

#Nov
#plt.ylim(10, 0)

#plt.xlim(1750, 1820)
#plt.ylim(9, 8)

#plt.xlim(5000, 7000)
#plt.ylim(10, 6)

plt.show(block=False)

print(len(STALTA_Points[0]))

- kNN Single File

In [ ]:
nn = 5
nbrs = NearestNeighbors(n_neighbors = nn)

kNN_Point = []

with tqdm(total=len(data[0])) as pbar:
    for samp_num in range(0, len(data[0])):
        outlier_locations = []
        if start_channel <= samp_num <= end_channel:

            time_sample = data.transpose()[samp_num,:]
            time_sample = np.array(time_sample)
            time_sample = time_sample.reshape(-1, 1)

            nbrs.fit(time_sample)

            distances, indexes = nbrs.kneighbors(time_sample)

            distances_mean = distances.mean(axis =1)
            th = distances_mean.mean() + (threshold * distances_mean.std())
            outlier_index = []
            for i, d in enumerate(distances_mean):
                if d > th:
                    outlier_index.append([i,d])

            outlier_index = np.array(outlier_index)
            outlier_locations = outlier_index[:,0].tolist()

        if outlier_locations != []:
            for l in outlier_locations:
                kNN_Point.append([samp_num, l])
        pbar.update()

kNN_Point = np.array(kNN_Point)

In [ ]:
bounds = 1000
fig, ax = plt.subplots()
img1 = ax.imshow(data, aspect='auto', interpolation='none', vmin=-bounds, vmax=bounds, extent=(first_channel,last_channel-1, ((last_time_sample - 1)/fs), (first_time_sample/fs)))
plt.set_cmap(cm.get_cmap('bwr'))
#ax.set_xlim(1200, 1500)
plt.ylabel('Time (seconds)')

ax.plot(kNN_Point[::, 0], kNN_Point[::, 1]/fs, color='black', linewidth = 3, marker='o', markerfacecolor='black', markersize=1, linestyle='None', label="Anomaly")

plt.ylabel('Time (seconds)')
plt.xlabel('Distance (Channels)')
fig.colorbar(img1, label= "Nano Strain per Second [nm/m/s]")

#location of different events of interest
#Jan
#plt.xlim(1500, 1700)

#plt.xlim(2500, 2800)
#plt.ylim(3.4, 3.8)

#Feb

#plt.xlim(2840, 2920)
#plt.ylim(10, 9)

#Nov
#plt.ylim(10, 0)

#plt.xlim(1750, 1820)
#plt.ylim(9, 8)

#plt.xlim(5000, 7000)
#plt.ylim(10, 6)

plt.show(block=False)

print(len(kNN_Point[::, 1]))

- kNN Multiple File, As the processing take hours we have saved out the extracted points for loading in

In [ ]:
#replace Xday for the data from other windows: xday filtered = 100Hz highpass, xday = no filter 
# path = "kNN Anoms/NDay/"
path = "kNN Anoms/NDay Filtered/"

#params
nn = 5
threshold = 2

#load data
with open(f"{path}knnAnomsTest", "rb") as fp:   # Unpickling
    e = pickle.load(fp)

with open(f"{path}knnAnomsTest2", "rb") as fp:   # Unpickling
    f = pickle.load(fp)

with open(f"{path}knnAnomsTest3", "rb") as fp:   # Unpickling
    g = pickle.load(fp)

e = e + f + g

timestart = None
timeend = None

#comment depending on window to extract correct period of data

#NDay
starttime = datetime.datetime(2023, 11, 9, 13, 39, 47)
graphtime = datetime.datetime(2023, 11, 9, 13, 49, 47)
length = 10

#JDay
#starttime = datetime.datetime(2024, 1, 17, 13, 00, 7)
#graphtime = datetime.datetime(2024, 1, 17, 13, 1, 37)
#length = 10

#FDay
# starttime = datetime.datetime(2024, 2, 27, 10, 30, 23)
# graphtime = datetime.datetime(2024, 2, 27, 10, 38, 23)
# length = 10


fs = 1000
startvalue = (graphtime - starttime).total_seconds() * fs

triggers_e = []

for i, channel in enumerate(e):
    for trigger_pair in channel[0]:
        if (startvalue + length * fs) >= trigger_pair > startvalue:# and trigger_pair[1] - trigger_pair[0] <= 10000:
            triggers_e.append([i, int(trigger_pair - startvalue)])

triggers_e = np.array(triggers_e)

temp_x = []
temp_y = []

if timeend is not None and timestart is not None:
    for x, y in zip(triggers_e[::, 0], triggers_e[::, 1]):
        if start_channel < x < end_channel and timestart <= y <= timeend:
            temp_x.append(x)
            temp_y.append(y)
else:
    for x, y in zip(triggers_e[::, 0], triggers_e[::, 1]):
        if start_channel < x < end_channel:
            temp_x.append(x)
            temp_y.append(y)

triggers_e = []
triggers_e.append(temp_x)
triggers_e.append(temp_y)
triggers_e = np.array(triggers_e)

In [ ]:
fig, ax = plt.subplots()
img1 = ax.imshow(data, aspect='auto', interpolation='none', extent=(first_channel,last_channel-1, ((last_time_sample - 1)/fs), (first_time_sample/fs)), vmin=-bounds, vmax=bounds)
#ax.set_xlim(1200, 1500)
plt.ylabel('Time (seconds)')

ax.plot(triggers_e[0], triggers_e[1]/fs, color='black', linewidth = 3, marker='o', markerfacecolor='black', markersize=1, linestyle='None', label="Anomaly")

plt.ylabel('Time (seconds)')
plt.xlabel('Distance (Channels)')
fig.colorbar(img1, label= "Nano Strain per Second [nm/m/s]")

#location of different events of interest
#Jan
#plt.xlim(1500, 1700)

#plt.xlim(2500, 2800)
#plt.ylim(3.4, 3.8)

#Feb

#plt.xlim(2840, 2920)
#plt.ylim(10, 9)

#Nov
#plt.ylim(10, 0)

#plt.xlim(1750, 1820)
#plt.ylim(9, 8)

#plt.xlim(5000, 7000)
#plt.ylim(10, 6)

plt.show(block=False)
plt.close(fig)

print(len(triggers_e[1]))

- Amplitude Thresholding

In [ ]:
#threshold for data
multiplier = 4
posData = abs(data)

Cropped_Raw_Points = np.where(posData > posData[:, start_channel:end_channel].mean() + multiplier * posData[:, start_channel:end_channel].std())
print(Cropped_Raw_Points)

temp_x = []
temp_y = []
for y, x in zip(Cropped_Raw_Points[1], Cropped_Raw_Points[0]):
    if start_channel <= y <= end_channel:
        temp_x.append(x)
        temp_y.append(y)

Cropped_Raw_Points = []
Cropped_Raw_Points.append(temp_x)
Cropped_Raw_Points.append(temp_y)
Cropped_Raw_Points = np.array(Cropped_Raw_Points)

In [ ]:
bounds = 1000
fig, ax = plt.subplots()
img1 = ax.imshow(data, aspect='auto', interpolation='none', vmin=-bounds, vmax=bounds , extent=(first_channel,last_channel-1, ((last_time_sample - 1)/fs), (first_time_sample/fs)))
plt.set_cmap(cm.get_cmap('bwr'))

ax.plot(Cropped_Raw_Points[1], Cropped_Raw_Points[0]/fs, color='black', linewidth = 3, marker='o', markerfacecolor='black', markersize=1, linestyle='None', label="Anomaly")

plt.ylabel('Time (seconds)')
plt.xlabel('Distance (Channels)')
fig.colorbar(img1, label= "Nano Strain per Second [nm/m/s]")

#location of different events of interest
#Jan
#plt.xlim(1500, 1700)

#plt.xlim(2500, 2800)
#plt.ylim(3.4, 3.8)

#Feb

#plt.xlim(2840, 2920)
#plt.ylim(10, 9)

#Nov
#plt.ylim(10, 0)

#plt.xlim(1750, 1820)
#plt.ylim(9, 8)

#plt.xlim(5000, 7000)
#plt.ylim(10, 6)

plt.show(block=False)

print(len(Cropped_Raw_Points[1]))

Section 4: Clustering Points

1. Select point dataset for clustering

In [ ]:
#ssta
point_data = list(zip(SSTA_Points[1], SSTA_Points[0]))
x = SSTA_Points

#sta/lta
#point_data = list(zip(STALTA_Points[1], STALTA_Points[0]))
#x=STALTA_Points

#knn
# point_data = list(zip(triggers_e[1], triggers_e[0]))
# x = []
# x.append(triggers_e[1])
# x.append(triggers_e[0])

#thresholding
#point_data = list(zip(Cropped_Raw_Points[1], Cropped_Raw_Points[0]))
#x=Cropped_Raw_Points

2. Cluster points

In [ ]:
point_data = np.array(point_data)

hdb = HDBSCAN(min_cluster_size= 4, min_samples=None)
clusters = hdb.fit_predict(point_data)

3. Plot Clusters as points

In [ ]:
bounds = 1000
fig, ax = plt.subplots()
img1 = ax.imshow(data, aspect='auto', interpolation='none', extent=(first_channel, last_channel-1, ((last_time_sample - 1)/fs), (first_time_sample/fs)), vmin=-bounds, vmax=bounds)
#ax.set_xlim(1200, 1500)
plt.ylabel('Time (seconds)')
img1.set_cmap(plt.cm.get_cmap('bwr'))

plt.set_cmap(plt.cm.get_cmap('gist_rainbow'))
ax.scatter(x[1], x[0]/fs, c=hdb.labels_, label="Anomaly", s = 2)
#plt.xlim(1750, 1950)
#plt.ylim(9, 8)

plt.show(block=False)
plt.close()

# Print cluster labels
print(max(clusters))

4. Plot clusters as windows (filter out windows with certain properties)

In [ ]:
#extract cluster window information from the point dataset
cluster_vals = []
for i in range(0, (max(clusters) + 1)):
    cluster_vals.append([])
tx = np.transpose(x)

for point, l in zip(tx, hdb.labels_):
    #print(point, l)
    cluster_vals[l].append(point)

#cluster_vals[0] = np.transpose(cluster_vals[0])
#cluster_vals[1] = np.transpose(cluster_vals[1])
#print(cluster_vals)
cluster_info = []
for c in cluster_vals:
    c = np.transpose(c)
    info = []
    #time
    #print(np.size(c))
    info.append(np.min(c[0]))
    info.append(np.max(c[0]))

    #channel
    info.append(np.min(c[1]))
    info.append(np.max(c[1]))
    info.append(np.shape(c)[1])

    cluster_info.append(info)

#plot the cluster windows
#unfiltered clusters
plot_clusters(some_data, cluster_info)

#clusters filtered by properties
plot_clusters(some_data, cluster_info, min_width= 12, max_width=100, num_points = 100)

5. Single Event Cluster Plotting

In [ ]:
windowData = []
count = 0

for c in cluster_info:
    diffY = c[1]-c[0]
    diffX = c[3]-c[2]
    
    #realtive window sizing
    # minY = c[0] - int(diffY)
    # if minY < 0:
    #     minY = 0
    # maxY = c[1] + int(diffY)
    #
    #
    # minX = c[2] - int(diffX) * 2
    # maxX = c[3] + int(diffX) * 2

    #absolute window sizing
    minY = c[0] + int(int(diffY)/2) - 50
    if minY < 0:
        minY = 0
    maxY = c[0] + int(int(diffY)/2) + 50

    minX = c[2] + int(int(diffX)/2) - 50
    maxX = c[2] + int(int(diffX)/2) + 50

    if 0 < diffX < 50 and c[4] > 10:
        windowData.append(data[minY:maxY, minX: maxX])

        bounds = 1000
        fig, ax = plt.subplots()
        img1 = ax.imshow(data, aspect='auto', interpolation='none', vmin=-bounds, vmax=bounds)
        #ax.set_xlim(1200, 1500)
        img1.set_cmap(plt.cm.get_cmap('bwr'))

        rect = patches.Rectangle((c[2], c[0]), c[3]- c[2], (c[1]-c[0]), linewidth=2, edgecolor="black", facecolor='none')
        ax.add_patch(rect)
        plt.ylim(maxY, minY)
        plt.xlim(minX, maxX)
        plt.show(block=False)
        plt.close()

        bounds = 1000
        fig, ax = plt.subplots()
        img1 = ax.imshow(windowData[count], aspect='auto', interpolation='none', vmin=-bounds, vmax=bounds)
        plt.ylabel('Time (milliseconds)')
        plt.title(f'Window {count}')
        img1.set_cmap(plt.colormaps['bwr'])
        plt.show(block=False)
        plt.close()
        count += 1

Section 5: Earthquakes

1. Load the Earthquake Data

In [ ]:
file_path = "Earthquake Data/Cromer_Earthquake_UTC_20250126_043223.140.tdms"

print('File: {0}'.format(file_path))

tdms = TdmsReader(file_path)

props = tdms.get_properties()

#where does data recording start
zero_offset = props.get('Zero Offset (m)')
#where does each channel sit along the cable
channel_spacing = props.get('SpatialResolution[m]') * props.get('Fibre Length Multiplier')
#how many channels are there
n_channels = tdms.fileinfo['n_channels']
#distance along the cable called depth here but hey
depth = zero_offset + np.arange(n_channels) * channel_spacing
#sampling frequency
fs = props.get('SamplingFrequency[Hz]')
time = props.get('GPSTimeStamp')

print('Number of channels in file: {0}'.format(n_channels))
print('Time samples in file: {0}'.format(tdms.channel_length))
print('Sampling frequency (Hz): {0}'.format(fs))
print(f'Time of Recording: {time}')

first_channel = 0
#If you want to read to the end get the channel length minus one
last_channel = n_channels
first_time_sample = 0
last_time_sample = tdms.channel_length - 1


some_data = tdms.get_data(first_channel, last_channel, first_time_sample, last_time_sample)
print('Size of data loaded: {0}'.format(some_data.shape))

from TDMS_Utilities import scale
some_data = scale(some_data, props)

In [ ]:
tdms2 = TdmsReader("Earthquake Data/Cromer_Earthquake_UTC_20250126_043253.140.tdms")
tdms3 = TdmsReader("Earthquake Data/Cromer_Earthquake_UTC_20250126_043323.140.tdms")

part1 = get_data(tdms)
part2 = get_data(tdms2)
part3 = get_data(tdms3)

big = np.append(part1, part2, axis=0)
data = np.append(big, part3, axis=0)

first_channel = 0
#If you want to read to the end get the channel length minus one
last_channel = n_channels
first_time_sample = 0
last_time_sample = 9000

2. Plot the data

In [ ]:
bounds = 1000

fig, ax = plt.subplots()
img1 = ax.imshow(data, aspect='auto', interpolation='none', extent=(first_channel,last_channel-1, ((last_time_sample - 1)/fs), (first_time_sample/fs)), vmin=-bounds, vmax=bounds)
plt.ylabel('Time (seconds)')

#ax.plot(depth[6000], 6250/fs, color='green', linewidth = 3, marker='o', markerfacecolor='green', markersize=5)

plt.xlabel('Distance (Metres)')
#plt.title(f"{props.get('GPSTimeStamp')} - Zoomed")
plt.title(f"Cromer Earthquake - {props.get('GPSTimeStamp')}\nRaw Waterfall")
plt.set_cmap(plt.cm.get_cmap('bwr'))
fig.colorbar(img1, label= "Nano Strain per Second [nm/m/s]")

plt.show(block=False)

3. Process the data using event detection methods

In [ ]:
filtered = False
start_channel = 450
end_channel = 1560

- SSTA

In [ ]:
num = 100
threshold = 6

mask = ssta(data, start_channel=start_channel, end_channel=end_channel, thresh=threshold, num=num)
SSTA_Points = np.where(mask == 1)
print(SSTA_Points)

In [ ]:
bounds = 1000
fig, ax = plt.subplots()
img1 = ax.imshow(data, aspect='auto', interpolation='none', vmin=-bounds, vmax=bounds, extent=(first_channel,last_channel-1, ((last_time_sample - 1)/fs), (first_time_sample/fs)))
plt.set_cmap(cm.get_cmap('bwr'))

ax.plot(SSTA_Points[1], SSTA_Points[0]/fs, color='black', linewidth = 3, marker='o', markerfacecolor='red', markersize=1, linestyle='None', label="Anomaly")

plt.ylabel('Time (seconds)')
plt.xlabel('Distance (Channels)')
fig.colorbar(img1, label= "Nano Strain per Second [nm/m/s]")
plt.show(block=False)

print(len(SSTA_Points[0]))

- STA/LTA

In [ ]:
lta = 10
sta = 0.1

processed_tdms = np.empty((last_time_sample+1, last_channel), type(data[0, 0]))

with tqdm(total=len(data[0])) as pbar:
    for samp_num in range(0, len(data[0])):
        #print(samp_num)
        if start_channel <= samp_num <= end_channel:
            time_sample = data.transpose()[samp_num,:]
            trace = Trace(np.array(time_sample))

            filtered_signal = classic_sta_lta(time_sample, int(sta * fs), int(lta * fs))

            for i, point in enumerate(filtered_signal):
                processed_tdms[i, samp_num] = point

        pbar.update()

STALTA_Points = np.where(processed_tdms > processed_tdms.mean() + 6 * processed_tdms.std())
print(STALTA_Points)

In [ ]:
bounds = 1000
plt.set_cmap(cm.get_cmap('bwr'))
fig, ax = plt.subplots()
img1 = ax.imshow(data, aspect='auto', interpolation='none', vmin=-bounds, vmax=bounds, extent=(first_channel,last_channel-1, ((last_time_sample - 1)/fs), (first_time_sample/fs)))
#ax.set_xlim(1200, 1500)
plt.ylabel('Time (seconds)')

ax.plot(STALTA_Points[1], STALTA_Points[0]/fs, color='black', linewidth = 3, marker='o', markerfacecolor='black', markersize=1, linestyle='None', label="Anomaly")

plt.ylabel('Time (seconds)')
plt.xlabel('Distance (Channels)')
fig.colorbar(img1, label= "Nano Strain per Second [nm/m/s]")


#plt.title(f"Cromer Earthquake - {props.get('GPSTimeStamp')}\nRaw Waterfall, STA/LTA\nSTA: {sta}s LTA: {lta}s Threshold: {threshold} std.dev")

plt.show(block=False)

print(len(STALTA_Points[0]))